# Web Scraping Student
## For: Michel Souza Santana
> Date start: 01/02/2022

* The purpose of this is to record the process of learning the WebScraping technique and make it available to anyone who wants to make use of it for their own development.

#### Instalando a biblioteca BeautifulSoup

In [2]:
# !pip install bs4

#### Importando as bibliotecas

In [38]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import requests
import pandas as pd
import numpy as np
import sqlite3

#### Coletando dados on-line via URL

> Fontes dos dados coletados
* https://www.adimimoveis.com.br/imoveis/a-venda 

##### OBS: A parte final desse código salva todo conteúdo em um arquivo html, assim sendo não será necessário rodar esse código a cada vêz para carregar os dados pois este será carregado a partirdo arquivo salvo polpando tempo em casos de consultas prolongadas. 

#### Por meio da url estarei capturando a quantidade de páginas existente no site para dai então percorrer uma a uma coletando os valores de meu interesse.

In [4]:
url="https://www.adimimoveis.com.br/imoveis/a-venda"
lista_paginacao = []
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
html = urlopen(req)
bs = BeautifulSoup(html.read(), 'html.parser')
lista_paginacao.append(bs.find_all('div', {'class':'pagination-cell hidden-lg-up'}))
lista_paginacao

[[<div class="pagination-cell hidden-lg-up"><p>1 de 50</p></div>]]

#### Armazenando o numero de páginas em uma variável

In [5]:
for i in lista_paginacao:
    number = []
    number.append(i[0].find('p').get_text())
number = number[0].split()
number_page = int(number[2])
number_page

50

#### Coletando os dados via URL

In [6]:
url="https://www.adimimoveis.com.br/imoveis/a-venda"
html_list = []
for page in range(1, number_page + 1):
    lista = []
    req = Request(url + '?pagina=' + str(page), headers={'User-Agent': 'Mozilla/5.0'})
    html = urlopen(req)
    bs = BeautifulSoup(html.read(), 'html.parser')
    lista.append(bs.find_all('div', {'class':'card_split_vertically__content'}))
    html_list.append(lista)

arq_html = open('imobiliario.html', 'w')
arq_html.write(str(html_list))
arq_html.close()

#### Selecionando os dados de interesse para análise dos valores dos imóveis em Balneário Camboriú

In [7]:
tabela = []
for dados in html_list:
    for cont in range(len(dados[0])):
        tabela.append(dados[0][cont])
tabela[0]

<div class="card_split_vertically__content"><div class="card_split_vertically__header"><p class="card_split_vertically__type">Apartamento</p><h3 class="card_split_vertically__location">Praia dos Amores - Itajaí - SC</h3><ul class="card_split_vertically__specs"><li class="card_split_vertically__spec">203.53 m²</li><li class="card_split_vertically__spec">4 Quartos</li><li class="card_split_vertically__spec">4 Suítes</li><li class="card_split_vertically__spec">1 Banheiro</li><li class="card_split_vertically__spec">2 Vagas</li></ul></div><div class="card_split_vertically__footer"><div class="card_split_vertically__value-container"><p class="card_split_vertically__value-title">Venda</p><p class="card_split_vertically__value">R$ 7.200.000</p></div></div></div>

#### Coletando os textos de dentro das TAGS

In [8]:
new_table = []
for i in tabela:
    new_list = []
    new_list.append(i.find('p', {'class': 'card_split_vertically__type'}).text) # Tipo do imóvel
    new_list.append(i.find('h3', {'class': 'card_split_vertically__location'}).text) # Endereço
    li=i.find_all('li', {'class': 'card_split_vertically__spec'}) # m2, Quartos, Suítes e Vagas
    for l in range(len(li)):
        new_list.append(li[l].text) # m2, Quartos, Suítes e Vagas
    new_list.append(i.find('p', {'class':'card_split_vertically__value-title'}).text) # Tipo de tramite
    new_list.append(i.find('p', {'class':'card_split_vertically__value'}).text)
    new_table.append(new_list)  
new_table[:3]  

[['Apartamento',
  'Praia dos Amores - Itajaí - SC',
  '203.53 m²',
  '4 Quartos',
  '4 Suítes',
  '1 Banheiro',
  '2 Vagas',
  'Venda',
  'R$ 7.200.000'],
 ['Apartamento',
  'Praia Brava - Itajaí - SC',
  '185.12 m²',
  '3 Quartos',
  '3 Suítes',
  '1 Banheiro',
  '3 Vagas',
  'Venda',
  'R$ 5.926.273,63'],
 ['Apartamento',
  'Praia Brava - Itajaí - SC',
  '210 m²',
  '4 Quartos',
  '4 Suítes',
  '1 Banheiro',
  '4 Vagas',
  'Venda',
  'R$ 6.089.548,34']]

#### Criando o DataFrame

In [9]:
df = pd.DataFrame(new_table)
df.head(10)

,0,1,2,3,4,5,6,7,8
0,Apartamento,Praia dos Amores - Itajaí - SC,203.53 m²,4 Quartos,4 Suítes,1 Banheiro,2 Vagas,Venda,R$ 7.200.000
1,Apartamento,Praia Brava - Itajaí - SC,185.12 m²,3 Quartos,3 Suítes,1 Banheiro,3 Vagas,Venda,"R$ 5.926.273,63"
2,Apartamento,Praia Brava - Itajaí - SC,210 m²,4 Quartos,4 Suítes,1 Banheiro,4 Vagas,Venda,"R$ 6.089.548,34"
3,Apartamento,Centro - Balneário Camboriú - SC,317.1 m²,4 Quartos,2 Suítes,1 Banheiro,2 Vagas,Venda,R$ 3.300.000
4,Apartamento,Centro - Balneário Camboriú - SC,185.63 m²,3 Quartos,3 Suítes,4 Banheiros,4 Vagas,Venda,R$ 6.950.000
5,Casa,Praia do Estaleiro - Balneário Camboriú - SC,458 m²,5 Quartos,4 Suítes,1 Banheiro,4 Vagas,Venda,R$ 20.000.000
6,Casa,Praia do Estaleiro - Balneário Camboriú - SC,458 m²,5 Quartos,4 Suítes,1 Banheiro,Venda,R$ 20.000.000,None
7,Apartamento,Centro - Balneário Camboriú - SC,190.17 m²,3 Quartos,3 Suítes,4 Banheiros,2 Vagas,Venda,R$ 1.980.000
8,Apartamento,Centro - Balneário Camboriú - SC,68.83 m²,2 Quartos,1 Suíte,2 Banheiros,1 Vaga,Venda,R$ 950.000
9,Apartamento,Pioneiros - Balneário Camboriú - SC,126.85 m²,3 Quartos,3 Suítes,1 Banheiro,2 Vagas,Venda,R$ 2.550.000


#### Verificando o tamanho do DF

In [10]:
df.shape

(592, 9)

#### Analisando as informações iniciais dos dados colhidos

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       592 non-null    object
 1   1       592 non-null    object
 2   2       592 non-null    object
 3   3       592 non-null    object
 4   4       592 non-null    object
 5   5       586 non-null    object
 6   6       555 non-null    object
 7   7       526 non-null    object
 8   8       483 non-null    object
dtypes: object(9)
memory usage: 41.8+ KB


#### Tratando os valores nulos

In [12]:
df.isnull().sum()

0      0
1      0
2      0
3      0
4      0
5      6
6     37
7     66
8    109
dtype: int64

Eliminando os NaN da tabela

In [13]:
df.dropna(axis=0, inplace=True)
df.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64

In [14]:
df.fillna(df, inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8
0,Apartamento,Praia dos Amores - Itajaí - SC,203.53 m²,4 Quartos,4 Suítes,1 Banheiro,2 Vagas,Venda,R$ 7.200.000
1,Apartamento,Praia Brava - Itajaí - SC,185.12 m²,3 Quartos,3 Suítes,1 Banheiro,3 Vagas,Venda,"R$ 5.926.273,63"
2,Apartamento,Praia Brava - Itajaí - SC,210 m²,4 Quartos,4 Suítes,1 Banheiro,4 Vagas,Venda,"R$ 6.089.548,34"
3,Apartamento,Centro - Balneário Camboriú - SC,317.1 m²,4 Quartos,2 Suítes,1 Banheiro,2 Vagas,Venda,R$ 3.300.000
4,Apartamento,Centro - Balneário Camboriú - SC,185.63 m²,3 Quartos,3 Suítes,4 Banheiros,4 Vagas,Venda,R$ 6.950.000


#### Iniciando o processo de limpeza das Strings 

In [15]:
df[8] = df[8].str.replace('[R$ ]', '')

/tmp/ipykernel_45163/1071970549.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df[8] = df[8].str.replace('[R$ ]', '')


In [16]:
df[2] = df[2].map(lambda x: x.split()[0])
df[3] = df[3].map(lambda x: x.split()[0])
df[4] = df[4].map(lambda x: x.split()[0])
df[5] = df[5].map(lambda x: x.split()[0])
df[6] = df[6].map(lambda x: x.split()[0])

In [17]:
new = df[1].str.split('-', expand=True)
df['bairro'] = new[0]
df['cidade'] = new[1]
df['estado'] = new[2]
df.drop(columns=[1], inplace=True)
df.head()

,0,2,3,4,5,6,7,8,bairro,cidade,estado
0,Apartamento,203.53,4,4,1,2,Venda,7.200.000,Praia dos Amores,Itajaí,SC
1,Apartamento,185.12,3,3,1,3,Venda,"5.926.273,63",Praia Brava,Itajaí,SC
2,Apartamento,210,4,4,1,4,Venda,"6.089.548,34",Praia Brava,Itajaí,SC
3,Apartamento,317.1,4,2,1,2,Venda,3.300.000,Centro,Balneário Camboriú,SC
4,Apartamento,185.63,3,3,4,4,Venda,6.950.000,Centro,Balneário Camboriú,SC


#### A coluna 7 refere-se ao contem valores do tipo de tranzação (vendas). Como todos tem o mesmo valor não precisamos desses dados, não terão peso em nossa analise.

In [18]:
df.drop(columns=[7], inplace=True)

#### Renomeando as colunas para facilitar o entendimento

In [19]:
df.rename(columns={0:'tipo_imovel', 2:'m2', 3:'quartos', 4:'suites', 5:'banheiros', 6:'vagas', 8:'preco'}, inplace=True)

#### Reorganizando a ordem das colunas para melhor assimilação

In [20]:
df = df[['cidade', 'bairro', 'estado', 'tipo_imovel', 'm2', 'quartos', 'suites', 'banheiros', 'vagas', 'preco']]
df.head()

,cidade,bairro,estado,tipo_imovel,m2,quartos,suites,banheiros,vagas,preco
0,Itajaí,Praia dos Amores,SC,Apartamento,203.53,4,4,1,2,7.200.000
1,Itajaí,Praia Brava,SC,Apartamento,185.12,3,3,1,3,"5.926.273,63"
2,Itajaí,Praia Brava,SC,Apartamento,210,4,4,1,4,"6.089.548,34"
3,Balneário Camboriú,Centro,SC,Apartamento,317.1,4,2,1,2,3.300.000
4,Balneário Camboriú,Centro,SC,Apartamento,185.63,3,3,4,4,6.950.000


#### Convertendo as colunas de valores numéricos para int e float de acordo com seus critérios

In [21]:
df['m2'] = df['m2'].map(lambda x: float(x))
df['quartos'] = df['quartos'].map(lambda x: int(x))
df['suites'] = df['suites'].map(lambda x: int(x))
df['banheiros'] = df['banheiros'].map(lambda x: int(x))
df['vagas'] = df['vagas'].map(lambda x: int(x))

In [22]:
df.head()

,cidade,bairro,estado,tipo_imovel,m2,quartos,suites,banheiros,vagas,preco
0,Itajaí,Praia dos Amores,SC,Apartamento,203.53,4,4,1,2,7.200.000
1,Itajaí,Praia Brava,SC,Apartamento,185.12,3,3,1,3,"5.926.273,63"
2,Itajaí,Praia Brava,SC,Apartamento,210.00,4,4,1,4,"6.089.548,34"
3,Balneário Camboriú,Centro,SC,Apartamento,317.10,4,2,1,2,3.300.000
4,Balneário Camboriú,Centro,SC,Apartamento,185.63,3,3,4,4,6.950.000


#### Limpando os dados da coluna "preco" pois nela encontram-se strings que impossibilitam a conversão para float.

In [23]:
df['preco'] = df['preco'].str.replace('[ .]', '')
df['preco'] = df['preco'].str.replace('[,]', '.')
df['preco'] = df['preco'].str.replace('Vendasobconsulta', '0')


/tmp/ipykernel_45163/7449221.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['preco'] = df['preco'].str.replace('[ .]', '')
/tmp/ipykernel_45163/7449221.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['preco'] = df['preco'].str.replace('[,]', '.')


In [24]:
df.head()

,cidade,bairro,estado,tipo_imovel,m2,quartos,suites,banheiros,vagas,preco
0,Itajaí,Praia dos Amores,SC,Apartamento,203.53,4,4,1,2,7200000
1,Itajaí,Praia Brava,SC,Apartamento,185.12,3,3,1,3,5926273.63
2,Itajaí,Praia Brava,SC,Apartamento,210.00,4,4,1,4,6089548.34
3,Balneário Camboriú,Centro,SC,Apartamento,317.10,4,2,1,2,3300000
4,Balneário Camboriú,Centro,SC,Apartamento,185.63,3,3,4,4,6950000


In [25]:
df['preco'] = df['preco'].map(lambda x: float(x))

In [26]:
df.head()

,cidade,bairro,estado,tipo_imovel,m2,quartos,suites,banheiros,vagas,preco
0,Itajaí,Praia dos Amores,SC,Apartamento,203.53,4,4,1,2,7200000.00
1,Itajaí,Praia Brava,SC,Apartamento,185.12,3,3,1,3,5926273.63
2,Itajaí,Praia Brava,SC,Apartamento,210.00,4,4,1,4,6089548.34
3,Balneário Camboriú,Centro,SC,Apartamento,317.10,4,2,1,2,3300000.00
4,Balneário Camboriú,Centro,SC,Apartamento,185.63,3,3,4,4,6950000.00


#### Verificando a quantia de cidades existentes na tabela

In [27]:
df['cidade'].unique()

array([' Itajaí ', ' Balneário Camboriú ', ' Camboriú ', ' Itapema '],
      dtype=object)

#### Conferindo a conversão dos tipos dos dados

In [28]:
df.dtypes

cidade          object
bairro          object
estado          object
tipo_imovel     object
m2             float64
quartos          int64
suites           int64
banheiros        int64
vagas            int64
preco          float64
dtype: object

#### Limpeza da Tabela Finalizada

In [29]:
df.head()

,cidade,bairro,estado,tipo_imovel,m2,quartos,suites,banheiros,vagas,preco
0,Itajaí,Praia dos Amores,SC,Apartamento,203.53,4,4,1,2,7200000.00
1,Itajaí,Praia Brava,SC,Apartamento,185.12,3,3,1,3,5926273.63
2,Itajaí,Praia Brava,SC,Apartamento,210.00,4,4,1,4,6089548.34
3,Balneário Camboriú,Centro,SC,Apartamento,317.10,4,2,1,2,3300000.00
4,Balneário Camboriú,Centro,SC,Apartamento,185.63,3,3,4,4,6950000.00


#### Salvando em um arquivo csv

In [34]:
df.to_csv('imobiliario.csv')